In [1]:
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt
import matplotlib as mpl 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

1.3.2


In [2]:
import sklearn 
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.2.


In [3]:
from platform import python_version
print(python_version())

3.8.11


# Load Data

In [4]:
Columns = [r'$c_1$', r'$c_2$',r'$N_s$',r'$A_s$',r'$U^{4+}$',r'$Na^+$','pH','pe', r'$logK_1$',
           r'$logK_2$',r'$logK_c$',r'$logK_a$', r'$logK_{UO_2^{2+}}$',r'$logK_{U^{4+}}$', 'pH2', 
           r'$\sigma_0$',r'$\sigma_{\beta}$',r'$\sigma_d$',r'$\psi_0$',r'$\psi_{\beta}$',r'$\psi_d$','A2', r'$\equiv SO^-$',
           r'$\equiv SOH_2^+$', r'$\equiv SOH_2^+:Cl^{-}$', r'$\equiv SO^-:UO_2^{2+}$',r'$\equiv SO^-:Na^+$',r'$\equiv SOH^0$',r'$\equiv SO^-:U^{4+}$']

In [5]:
df = pd.read_csv('dataset/DATABASE_AGAIN.txt', delimiter=r"\s+", low_memory=False, names = Columns)

In [6]:
df.pH.max(), df.pH.min(), df.shape

(12.0, 2.0, (1589411, 29))

In [7]:
# load x_train, x_val, x_test 
inputs = [r'$c_1$', r'$N_s$',r'$A_s$',r'$U^{4+}$',r'$Na^+$','pH', r'$logK_1$',
           r'$logK_2$',r'$logK_c$',r'$logK_a$', r'$logK_{UO_2^{2+}}$',r'$logK_{U^{4+}}$']

targets = [r'$\sigma_0$',r'$\sigma_{\beta}$',r'$\sigma_d$',r'$\psi_0$',r'$\psi_{\beta}$',r'$\psi_d$', r'$\equiv SO^-$',
           r'$\equiv SOH_2^+$', r'$\equiv SO^-:UO_2^{2+}$',r'$\equiv SO^-:Na^+$',r'$\equiv SOH^0$',r'$\equiv SO^-:U^{4+}$']

df_train_norm = pd.read_csv('dataset/train_norm.csv')
df_val_norm = pd.read_csv('dataset/val_norm.csv')
df_test_norm = pd.read_csv('dataset/test_norm.csv')

X_train_scaled = df_train_norm[inputs]
y_train_scaled = df_train_norm[targets]

X_val_scaled = df_val_norm[inputs]
y_val_scaled = df_val_norm[targets]

X_test_scaled = df_test_norm[inputs]
y_test_scaled = df_test_norm[targets]

In [8]:
df_train.shape, df_val.shape, df_test.shape, df.shape

((1271528, 24), (158941, 24), (158942, 24), (1589411, 29))

In [14]:
X_train.shape, X_val.shape, X_test.shape

((1271528, 12), (158941, 12), (158942, 12))

In [15]:
X_train_scaled = (X_train - X_train.min()) / (X_train.max() - X_train.min())
y_train_scaled = (y_train - y_train.min()) / (y_train.max() - y_train.min())

X_val_scaled = (X_val - X_train.min()) / (X_train.max() - X_train.min())
y_val_scaled = (y_val - y_train.min()) / (y_train.max() - y_train.min())

X_test_scaled = (X_test - X_train.min()) / (X_train.max() - X_train.min())
y_test_scaled = (y_test - y_train.min()) / (y_train.max() - y_train.min())

In [44]:
# save dataset
pd.concat([X_train, y_train], axis = 1).to_csv('dataset/train.csv', index=False)
pd.concat([X_val, y_val], axis = 1).to_csv('dataset/val.csv', index=False)
pd.concat([X_test, y_test], axis = 1).to_csv('dataset/test.csv', index=False)

In [45]:
pd.concat([X_train_scaled, y_train_scaled], axis = 1).to_csv('dataset/train_norm.csv', index=False)
pd.concat([X_val_scaled, y_val_scaled], axis = 1).to_csv('dataset/val_norm.csv', index=False)
pd.concat([X_test_scaled, y_test_scaled], axis = 1).to_csv('dataset/test_norm.csv', index=False)

In [51]:
# (pd.read_csv('dataset/test_norm.csv'))

# Train model using multiouput regressor 

In [11]:
from sklearn.multioutput import MultiOutputRegressor

In [37]:
import time 
start_time = time.perf_counter()
estimator_RF = RandomForestRegressor(n_estimators=100,
                                     oob_score=True, 
                                     random_state=20
                                    )
regr_RF = MultiOutputRegressor(estimator_RF).fit(X_train_scaled, y_train_scaled)
end_time = time.perf_counter()
print('training time is: ', end_time - start_time, 's')

training time is:  5144.91960903001 s


In [39]:
# predict on test data 
import time
start_time = time.perf_counter()
y_pred_multioutRF = regr_RF.predict(X_test_scaled)
end_time = time.perf_counter() 
print('RF test time is: ', end_time - start_time, 's')

RF test time is:  86.7091995239025 s


In [40]:
# save predicted data 
np.savetxt('res/y_pred_multioutRF_njobs1.txt', y_pred_multioutRF)

# Training on DNN 

In [11]:
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from model.pytorchtools import EarlyStopping
import model.net as models 
from model.dataset import SurfaceComplexationDataset
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

In [12]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
    return optimizer, scheduler

In [13]:
def load_data(data_dir): 
    train_set = SurfaceComplexationDataset(root_dir=data_dir, split = 'train')
    test_set = SurfaceComplexationDataset(root_dir=data_dir, split='test')
    val_set = SurfaceComplexationDataset(root_dir=data_dir, split='val')

    return train_set, val_set, test_set

In [14]:
def train_epoch(train_loader, model, optimizer, device, epoch):
    """ Train the model on num_steps batches 
    Args: 
        train_loader: a torch.utils.data.DataLoader object that fetches the data
        model: the neural network 
        optimizer: adams 
    """
    model.train()
    running_loss = 0.0
    num_batch = len(train_loader)

    for i, (inputs, targets) in enumerate(train_loader): 
        inputs, targets = inputs.to(device), targets.to(device)
        # zero the paramter gradients 
        optimizer.zero_grad()

        # forward + backward + optimize 
        pred = model(inputs)
        loss = F.mse_loss(pred, targets)
        loss.backward()
        optimizer.step()

        # print statistics 
        running_loss += loss.item()
        # if i % 300 == 0: 
        #     print('[%d: %d/%d] train loss: %f ' % (epoch, i, num_batch, loss.item()))
        # if i % 300 == 0: 
        #     print('[%d: %d/%d] train loss: %f lr = %f' % (epoch, i, num_batch, loss.item(), optimizer.param_groups[0]["lr"]))

    return running_loss / num_batch 

In [15]:
def validate(val_dataloader, model, device): 
    model.eval()
    val_running_loss = 0.0 

    with torch.no_grad(): 
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = F.mse_loss(outputs, targets)

            val_running_loss += loss.item() * inputs.size(0)

    return val_running_loss / len(val_dataloader.dataset)

In [16]:
def plot_pramas(test_y, test_pred, foldername, outfile): 
    # print("R2 of training is: ", r2_score(train_y, train_pred))
    np.savetxt(f'{foldername}/test_pred_{outfile}.txt', test_pred)
    np.savetxt(f'{foldername}/test_y_{outfile}.txt', test_y)
    
    print("R2 of test is: ", r2_score(test_y, test_pred))

    test_mse = mean_squared_error(test_y, test_pred)
    test_mae = mean_absolute_error(test_y, test_pred)

    print('Test set results for %i samples:' % test_pred.shape[0])
    print('MSE:', test_mse)
    print('MAE:', test_mae)

In [17]:
def test_accuracy(net, testloader, foldername, outfile, device): 
    test_pred = []
    test_y = []
    running_loss = 0 

    with torch.no_grad():
        for data in testloader:
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = F.mse_loss(outputs, targets)

            running_loss += loss.item() 

            pred_val_numpy = outputs.data.cpu().numpy()
            target_val_numpy = targets.data.cpu().numpy()

            test_pred.append(pred_val_numpy)
            test_y.append(target_val_numpy)

    test_pred = np.concatenate(test_pred, axis=0)
    test_y = np.concatenate(test_y, axis=0)

    plot_pramas(test_y, test_pred, foldername, outfile)

    print('MSE loss on test set is:', running_loss / len(testloader.dataset)) 

In [20]:
def train_model(model, device, train_loader, val_loader, test_loader, optimizer, lr_scheduler, isSch, res_dir, name, patience = 20, n_epochs = 100): 
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 

    blue = lambda x: '\033[94m' + x + '\033[0m'
    
    checkpoint_dir = os.path.join(res_dir, 'checkpoints')
    try:
        os.makedirs(res_dir)
        os.makedirs(checkpoint_dir)
    except OSError:
        pass

    checkpoint_path = os.path.join(checkpoint_dir, f'{name}.pt')
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True, path = checkpoint_path)

    for epoch in tqdm(range(1, n_epochs + 1)):
        ###################
        # train the model #
        ###################
        train_epoch_loss = train_epoch(train_loader, model, optimizer, device, epoch)
        val_epoch_loss = validate(val_loader, model, device)

        # print loss every epoch 
        print('[%d] train loss: %f ' % (epoch, train_epoch_loss))
        print('[%d] %s loss: %f' % (epoch, blue('validate'), val_epoch_loss))

        avg_train_losses.append(train_epoch_loss)
        avg_valid_losses.append(val_epoch_loss)
        
        if isSch: 
            lr_scheduler.step(val_epoch_loss) 
        
        # add early stopping 
        # early_stopping(val_epoch_loss, model)
        early_stopping(train_epoch_loss, model)
        if early_stopping.early_stop: 
            print("Early stopping")
            break 

    np.savetxt(os.path.join(res_dir, f'train_loss_{name}.csv'), avg_train_losses)
    np.savetxt(os.path.join(res_dir, f'val_loss_{name}.csv'), avg_valid_losses) 

    # load the last checkpoint with the best model
    model.load_state_dict(torch.load(checkpoint_path)) 

    # test on test set 
    test_accuracy(model, test_loader, res_dir, name, device)
    # print(optimizer.state_dict())

In [21]:
def train_main(config): 
    data_dir = 'dataset/'

    # get dataset 
    train_set, val_set, test_set = load_data(data_dir)

    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=4, 
        pin_memory=False)

    val_loader = torch.utils.data.DataLoader(
            val_set,
            batch_size=int(config["batch_size"]),
            shuffle=True,
            num_workers=4, 
            pin_memory=False)

    test_loader = torch.utils.data.DataLoader(
            test_set, 
            batch_size=int(config["batch_size"]), 
            shuffle=True,
            num_workers=4, 
            pin_memory=False)
            
    print("Creating model")
    Model = getattr(models, config['model'])
    print('created model is: ', Model)

    model = Model(config['batch_norm'], config['layer_norm'], 
                 config["l1"], config["l2"], config["l3"], config["l4"], config["l5"])

    name = f"{config['model']}_{config['l1']}{config['l2']}{config['l3']}{config['l4']}" +\
            f"{config['l5']}lr{config['lr']}BS{config['batch_size']}isB{config['batch_norm']}" +\
            f"ln{config['layer_norm']}Opt{config['optimizer']}sch{config['lr_scheduler']}"
        

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer, lr_scheduler = build_optimizer(model, config['optimizer'], config['lr'])
    res_dir = 'DNN_res/'
    
    train_model(model, device, train_loader, val_loader, test_loader, optimizer, lr_scheduler, config['lr_scheduler'], res_dir, name, 20, 5000)

## train model 

In [ ]:
config = {'l1': 512, 'l2': 512, 'l3': 512, 'l4': 512, 'l5': 512, 
          'lr': 0.001, 'batch_size': 128, 'model': 'DeepNet6LayerTune', 'batch_norm': False, 
          'layer_norm': True, 'lr_scheduler': True, 'optimizer': 'adam'}

In [ ]:
import time 
start_time = time.perf_counter()
train_main(config)
end_time = time.perf_counter() 
print("time used to train model with 20/5000 patience is: ", (end_time - start_time)/60, 'mins', (end_time - start_time) / 3600, 'hrs')

Creating model
created model is:  <class 'model.net.DeepNet6LayerTune'>


  0%|          | 1/5000 [00:31<44:21:11, 31.94s/it]

[1] train loss: 0.001919 
[1] validate loss: 0.000545
Validation loss decreased (inf --> 0.001919).  Saving model ...


  0%|          | 2/5000 [01:03<44:19:35, 31.93s/it]

[2] train loss: 0.000451 
[2] validate loss: 0.000282
Validation loss decreased (0.001919 --> 0.000451).  Saving model ...


...

  5%|▌         | 263/5000 [2:20:01<42:30:47, 32.31s/it]

[263] train loss: 0.000001 
[263] validate loss: 0.000001
EarlyStopping counter: 19 out of 20


  5%|▌         | 263/5000 [2:20:33<42:11:33, 32.07s/it]

[264] train loss: 0.000001 
[264] validate loss: 0.000001
EarlyStopping counter: 20 out of 20
Early stopping


R2 of test is:  0.9999242235127709
Test set results for 158942 samples:
MSE: 1.0583532e-06
MAE: 0.00058783824
MSE loss on test set is: 8.270826198923937e-09
time used to train model with 20/5000 patience is:  140.69916973256719 mins 2.3449861622094534 hrs


# test model performance with two additional test

### RF

In [25]:
df_random_scale = pd.read_csv('dataset/set1/train_norm.csv')

In [16]:
# RF regressor 
import joblib
regr_RF = joblib.load('newRF_multiOutRegressor_nest100_oobTrue_rand20_njobs1.joblib')

In [27]:
# predict on test data 
import time
start_time = time.perf_counter()
y_pred_set1_multioutRF = regr_RF.predict(df_random_scale[inputs])
end_time = time.perf_counter() 
print('RF test time is: ', end_time - start_time, 's')

RF test time is:  0.09720314387232065 s


### DNN

In [24]:
def predict_on_test(config, data_dir = 'dataset/set1/', testdata = 'set1'): 

    # get dataset 
    _, _, test_set = load_data(data_dir)

    test_loader = torch.utils.data.DataLoader(
            test_set, 
            batch_size=int(config["batch_size"]), 
            shuffle=True,
            num_workers=4, 
            pin_memory=False)
            
    print("Creating model")
    Model = getattr(models, config['model'])
    print('created model is: ', Model)
    
    model = Model(config['batch_norm'], config['layer_norm'], 
                     config["l1"], config["l2"], config["l3"], config["l4"], config["l5"])
        
    name = f"{config['model']}_{config['l1']}{config['l2']}{config['l3']}{config['l4']}{config['l5']}lr{config['lr']}BS{config['batch_size']}isB{config['batch_norm']}ln{config['layer_norm']}Opt{config['optimizer']}sch{config['lr_scheduler']}"
    

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer, lr_scheduler = build_optimizer(model, config['optimizer'], config['lr'])
    
    res_dir = 'DNN_res'
    checkpoint_dir = os.path.join(res_dir, 'checkpoints')
    checkpoint_path = os.path.join(checkpoint_dir, f'{name}.pt')
    
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load(checkpoint_path)) 

    # test on test set 
    test_accuracy(model, test_loader, res_dir, testdata+name, device) 

In [25]:
config = {'l1': 512, 'l2': 512, 'l3': 512, 'l4': 512, 'l5': 512, 
          'lr': 0.001, 'batch_size': 128, 'model': 'DeepNet6LayerTune', 'batch_norm': False, 
          'layer_norm': True, 'lr_scheduler': True, 'optimizer': 'adam'}

import time 
start_time = time.perf_counter()
predict_on_test(config, data_dir='dataset/', testdata='test')
end_time = time.perf_counter()
print('time used to evaluate the model on test data: ', end_time - start_time, 'sec', (end_time - start_time) / 60, 'mins')

Creating model
created model is:  <class 'model.net.DeepNet6LayerTune'>
R2 of test is:  0.9999466318557761
Test set results for 158942 samples:
MSE: 8.477168e-07
MAE: 0.00057548966
MSE loss on test set is: 6.624687552514079e-09
time used to evaluate the model on test data:  13.97365738498047 sec 0.2328942897496745 mins


In [27]:
config = {'l1': 512, 'l2': 512, 'l3': 512, 'l4': 512, 'l5': 512, 
          'lr': 0.001, 'batch_size': 128, 'model': 'DeepNet6LayerTune', 'batch_norm': False, 
          'layer_norm': True, 'lr_scheduler': True, 'optimizer': 'adam'}

import time 
predict_on_test(config, data_dir='dataset/set2/', testdata='set2')
# predict_on_test(config, data_dir='dataset/set1/', testdata='set1')

Creating model
created model is:  <class 'model.net.DeepNet6LayerTune'>
R2 of test is:  -0.7321651537955086
Test set results for 50 samples:
MSE: 0.04378244
MAE: 0.13140967
MSE loss on test set is: 0.000875648707151413
